# 전처리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
train = pd.read_csv("train.csv")

## 데이터 확인

In [ ]:
train

### 데이터 설명
1. Category : 범죄 유형 - Target 데이터
2. Dates : 발생 일자
3. Descript : 범죄 상세 설명 
4. DayOfWeek : 요일
5. PdDistrict : 관할구역(총 10개) 
6. Resolution : 해결여부 
7. Address : 범죄 발생 주소
8. X, Y : 범죄 발생 장소의 경도, 위도

- 데이터 개수 : 878,049개
- 특징 개수 : 9개

## 중복데이터 제거

In [ ]:
# 중복값 확인
train.duplicated().sum()

In [ ]:
# 중복값 제거
train = train.drop_duplicates().reset_index(drop=True)
train

## 이상치 제거

In [ ]:
# 위도, 경도 좌표로 그래프 그리기
plt.figure(figsize=(10,8))

sns.scatterplot(x=train['X'], y=train['Y'])

- 이상치로 인해 위의 그래프에서 샌프란시스코 지역을 볼 수 없음

In [ ]:
# 좌표에서 가장 큰 값(이상치)을 확인
train['X'].max(), train['Y'].max()

- 경도 -122를 벗어난 -120 이상치 발견

In [ ]:
# 경도 -122보다 낮은 값들만 추출
train = train[train['X'] < -122]
train.head()

In [ ]:
# 위도, 경도 좌표로 그래프 그리기
plt.figure(figsize=(10,8))

sns.scatterplot(x=train['X'], y=train['Y'])

## 결측치 확인

In [ ]:
train.isnull().sum()                    # 결측치 없음                 

## Dates 세분화

### Dates 피처 속성 변환 & 년,월,일,시,분 세분화

In [ ]:
# 컬럼별 데이터 타입 확인
train.dtypes

- Dates 컬럼 데이터 타입은 문자열이므로 datetime 형으로 변환
- 필요한 년, 월, 일, 시, 분, 초 추출 후 새로운 컬럼으로 추가

In [ ]:
# datetime으로 변경하는 함수 정의
def to_datetime(data):
    data['Dates'] = pd.to_datetime(data['Dates'])
    
    data['Year'] = data['Dates'].dt.year.astype('int')
    data['Month'] = data['Dates'].dt.month.astype('int')
    data['Day'] = data['Dates'].dt.day.astype('int')
    data['Hour'] = data['Dates'].dt.hour.astype('int')
    data['Minute'] = data['Dates'].dt.minute.astype('int')
    data['Second'] = data['Dates'].dt.second.astype('int')
    
    return data

In [ ]:
# 날짜 데이터 datetimes64 타입으로 변경
train = to_datetime(train.copy())

In [ ]:
# datetime으로 변경됐는지 확인
train.dtypes

### 평일, 주말 구분

In [ ]:
# train 데이터셋 주말, 평일 추출 함수 정의
def weekday_weekend(data):
    data['is_weeked'] = data['DayOfWeek'].apply(lambda x : 'weekend' if x == 'Saturday' or x =='Sunday' else 'weekday')
    
    return data

In [ ]:
# train 데이터셋 주말, 평일 추출후 컬럼 옆에 추가하기
train = weekday_weekend(train.copy())

### 계절 구분

In [ ]:
# 계절 컬럼 추가하는 함수 정의
def season(m):
  if 3 <= m <= 5:
    season = 'spring'
  elif 6 <= m <=8:
    season = 'summer'
  elif 9 <= m <= 11:
    season = 'fall'
  else:
    season = 'winter'

  return season

In [ ]:
# 계절 컬럼 추가하기
train['Season'] = train['Month'].apply(season)

In [ ]:
# 계절 추가된지 확인하기
train['Season'].unique()

## 추가된 컬럼 설명

In [ ]:
train.columns

Year    : Dates에서 년도만 추출<br>
Month   : Dates에서 월만 추출<br>
Day     : Dates에서 일만 추출<br>
Hour    : Dates에서 시간만 추출<br>
Minute  : Dates에서 분만 추출<br>
Second  : Dates에서 초만 추출<br>
is_weeked : DayOfWeek에서 Saturday 또는 Sunday 이면 'weekend'로 아니면 'weekday' 로 변경 <br>
Season  : 12월-2월 'winter', 3월-5월 'spring', 6월-8월 'summer', 9월-11월 'fall'
